In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import linearmodels.iv.model as lm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

In [2]:
df = pd.read_csv("Data-GP1-1.csv")
df

,Mon,Tue,Wed,Thu,Date,Stormy,Mixed,p,q,Rainy,Cold,Wind
0,1,0,0,0,911202,1,0,-0.430783,8.994421,1,0,2.995732
1,0,1,0,0,911203,1,0,0.000000,7.707063,0,0,2.995732
2,0,0,1,0,911204,0,1,0.072321,8.350194,1,1,2.813411
3,0,0,0,1,911205,1,0,0.247139,8.656955,0,1,3.036554
4,0,0,0,0,911206,1,0,0.664327,7.844241,0,1,3.036554
...,...,...,...,...,...,...,...,...,...,...,...,...
106,1,0,0,0,920504,0,0,-0.798508,8.610683,0,0,2.862201
107,0,1,0,0,920505,0,1,-0.087011,7.162397,0,0,2.908721
108,0,0,1,0,920506,0,1,0.184922,7.362010,0,0,2.862201
109,0,0,0,1,920507,0,1,0.223143,8.764053,0,0,2.813411


# Data Exploration - Correlation Analysis

## Overall 

In [3]:
df.corr().style.background_gradient(cmap='bwr_r', vmin = -1, vmax = 1)

,Mon,Tue,Wed,Thu,Date,Stormy,Mixed,p,q,Rainy,Cold,Wind
Mon,1.000000,-0.246951,-0.233333,-0.246951,-0.060778,0.098818,-0.121380,-0.074936,0.156964,-0.025299,0.018651,0.165220
Tue,-0.246951,1.000000,-0.246951,-0.261364,-0.036667,0.018126,0.046049,-0.018483,-0.218871,-0.104308,0.062080,0.117045
Wed,-0.233333,-0.246951,1.000000,-0.246951,0.057674,-0.053526,0.128124,-0.009366,-0.234749,0.037105,0.064657,-0.045354
Thu,-0.246951,-0.261364,-0.246951,1.000000,0.019542,-0.030946,-0.002172,0.083350,0.165607,-0.044005,-0.026835,-0.119755
Date,-0.060778,-0.036667,0.057674,0.019542,1.000000,-0.159531,0.021537,-0.244170,-0.066379,-0.102542,-0.168876,-0.186811
Stormy,0.098818,0.018126,-0.053526,-0.030946,-0.159531,1.000000,-0.422917,0.399417,-0.222636,0.097708,0.392061,0.753123
Mixed,-0.121380,0.046049,0.128124,-0.002172,0.021537,-0.422917,1.000000,0.065993,-0.008908,-0.080250,-0.084166,-0.046567
p,-0.074936,-0.018483,-0.009366,0.083350,-0.244170,0.399417,0.065993,1.000000,-0.278530,0.036653,0.241886,0.405123
q,0.156964,-0.218871,-0.234749,0.165607,-0.066379,-0.222636,-0.008908,-0.278530,1.000000,0.033979,-0.142820,-0.263995
Rainy,-0.025299,-0.104308,0.037105,-0.044005,-0.102542,0.097708,-0.080250,0.036653,0.033979,1.000000,0.093805,0.065632


## Correlation Analysis between q and other variables

In [4]:
q_corr =pd.DataFrame(df.corr()['q'])
q_corr.T.style.background_gradient(cmap='bwr_r', vmin = -1, vmax = 1)

,Mon,Tue,Wed,Thu,Date,Stormy,Mixed,p,q,Rainy,Cold,Wind
q,0.156964,-0.218871,-0.234749,0.165607,-0.066379,-0.222636,-0.008908,-0.278530,1.000000,0.033979,-0.142820,-0.263995


- Weak relationship: `Tue`, `Wed`, `Stormy`, `p`, `Wind`
- Others have nearly no relationship with `q`

## Correlation Analysis between p and other variables

In [5]:
p_corr =pd.DataFrame(df.corr()['p']).T
p_corr.style.background_gradient(cmap='bwr_r', vmin = -1, vmax = 1)

,Mon,Tue,Wed,Thu,Date,Stormy,Mixed,p,q,Rainy,Cold,Wind
p,-0.074936,-0.018483,-0.009366,0.083350,-0.244170,0.399417,0.065993,1.000000,-0.278530,0.036653,0.241886,0.405123


- Low correlation: `Date`, `q`, `Cold`
- Moderate correlation/Substantial relationship: `Stormy`, `Wind`
- We can hypothesize both `Stormy` and `Wind` can be the potential instrumental variables

## Correlation Analysis between Stormy and other variables

In [6]:
stormy_corr =pd.DataFrame(df.corr()['Stormy'])
stormy_corr.T.style.background_gradient(cmap='bwr_r', vmin = -1, vmax = 1)

,Mon,Tue,Wed,Thu,Date,Stormy,Mixed,p,q,Rainy,Cold,Wind
Stormy,0.098818,0.018126,-0.053526,-0.030946,-0.159531,1.000000,-0.422917,0.399417,-0.222636,0.097708,0.392061,0.753123


- Low correlation: `q`
- Moderate correlation: `Mixed`, `p`, `Cold`
- Strong correlation: `Wind`

## Correlation Analysis between Wind and other variables

In [7]:
wind_corr =pd.DataFrame(df.corr()['Wind'])
wind_corr.T.style.background_gradient(cmap='bwr_r', vmin = -1, vmax = 1)

,Mon,Tue,Wed,Thu,Date,Stormy,Mixed,p,q,Rainy,Cold,Wind
Wind,0.165220,0.117045,-0.045354,-0.119755,-0.186811,0.753123,-0.046567,0.405123,-0.263995,0.065632,0.450201,1.000000


- low correlation: `q`
- Moderate correlation: `p`, `Cold`
- Strong correlation: `Stormy`

## Correlation Among Weekdays

In [8]:
week_corr = df.corr()[:4].style.background_gradient(cmap='bwr_r', vmin = -1, vmax = 1)
week_corr

,Mon,Tue,Wed,Thu,Date,Stormy,Mixed,p,q,Rainy,Cold,Wind
Mon,1.000000,-0.246951,-0.233333,-0.246951,-0.060778,0.098818,-0.121380,-0.074936,0.156964,-0.025299,0.018651,0.165220
Tue,-0.246951,1.000000,-0.246951,-0.261364,-0.036667,0.018126,0.046049,-0.018483,-0.218871,-0.104308,0.062080,0.117045
Wed,-0.233333,-0.246951,1.000000,-0.246951,0.057674,-0.053526,0.128124,-0.009366,-0.234749,0.037105,0.064657,-0.045354
Thu,-0.246951,-0.261364,-0.246951,1.000000,0.019542,-0.030946,-0.002172,0.083350,0.165607,-0.044005,-0.026835,-0.119755


- Weekday are definitely interdependent on one another because of the nature of binary variables

## Findings:
- To prevent multi-collinearity, `Wind` and `Stormy` should not be used together
- `Mixed`, `Cold`  may not be good controls due to their moderate correlations with `Stormy` and `Wind`

# OLS Regression

## Simple Linear Regression as the Baseline Demand

In [9]:
y = df['q']
X = df['p']
X = sm.add_constant(X)
baseline_model = sm.OLS(y,X).fit()
print(baseline_model.summary())

                            OLS Regression Results                            
Dep. Variable:                      q   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     9.167
Date:                Fri, 22 Sep 2023   Prob (F-statistic):            0.00308
Time:                        15:31:05   Log-Likelihood:                -119.35
No. Observations:                 111   AIC:                             242.7
Df Residuals:                     109   BIC:                             248.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.4187      0.076    110.445      0.0

- Although price is significant with p-value <0.05, low R-squared value indicates that solely price is not explaining much in the variation of the quantity

## Multilinear regression for Control Variable Selection

In [10]:
#Baseline
m1 = smf.ols('q ~ p',data=df, ).fit()

#Based on correlation analysis
m2 = smf.ols('q ~ p + Stormy  + Tue + Wed ',data=df).fit()

#Only stormy to prevent multicollinearity
m3 = smf.ols('q ~ p + Stormy  + Mon + Tue + Wed + Thu + Mixed + Rainy + Cold',data=df).fit()

#Include all the variables in the dataset
m4 = smf.ols('q ~ p + Stormy + Wind + Mon + Tue + Wed + Thu + Mixed + Rainy + Cold',data=df).fit()

print("Dependent variable: q", summary_col([m1,m2, m3, m4], float_format='%.3f',stars=True,regressor_order=['Intercept', 'p', 'Stormy', 'Wind', 'Mon', 'Tue', 'Wed', 'Thu', 'Mixed', 'Rainy', 'Cold'], model_names = ['I', 'II', 'III', 'IV']))

Dependent variable: q 
                   I         II       III        IV   
------------------------------------------------------
Intercept      8.419***  8.730***  8.665***  10.114***
               (0.076)   (0.104)   (0.181)   (1.529)  
p              -0.541*** -0.447**  -0.449**  -0.434** 
               (0.179)   (0.180)   (0.195)   (0.195)  
Stormy                   -0.232    -0.241    -0.044   
                         (0.151)   (0.189)   (0.280)  
Wind                                         -0.543   
                                             (0.570)  
Mon                                0.061     0.103    
                                   (0.207)   (0.212)  
Tue                      -0.545*** -0.489**  -0.458** 
                         (0.160)   (0.204)   (0.207)  
Wed                      -0.600*** -0.554*** -0.553***
                         (0.165)   (0.208)   (0.208)  
Thu                                0.087     0.079    
                                   (0.201)

# 2SLS Regression

## The baseline & statistically best model : Stormy as IV without any control

In [14]:
y = df['p']
X = df['Stormy']
X = sm.add_constant(X)
baseline_stage1 = sm.OLS(y,X).fit()
print(baseline_stage1.summary())

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.152
Method:                 Least Squares   F-statistic:                     20.69
Date:                Fri, 22 Sep 2023   Prob (F-statistic):           1.41e-05
Time:                        15:39:56   Log-Likelihood:                -40.516
No. Observations:                 111   AIC:                             85.03
Df Residuals:                     109   BIC:                             90.45
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2903      0.040     -7.336      0.0

In [15]:
p_hat = baseline_stage1.predict(X)
df['p_hat'] = p_hat

In [16]:
y = df['q']
X = df['p_hat']
X = sm.add_constant(X)
baseline_stage2 = sm.OLS(y,X).fit()
print(baseline_stage2.summary())

                            OLS Regression Results                            
Dep. Variable:                      q   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     5.685
Date:                Fri, 22 Sep 2023   Prob (F-statistic):             0.0188
Time:                        15:40:01   Log-Likelihood:                -121.01
No. Observations:                 111   AIC:                             246.0
Df Residuals:                     109   BIC:                             251.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.3138      0.112     74.406      0.0

## The more generalized model: Stormy as IV, all Weekdays as Controls

In [30]:
Z = df['p']
X = df[['Stormy', 'Mon', 'Tue', 'Wed', 'Thu']]
X = sm.add_constant(X)
gen_stage1 = sm.OLS(Z,X).fit()
print(gen_stage1.summary())

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.179
Model:                            OLS   Adj. R-squared:                  0.140
Method:                 Least Squares   F-statistic:                     4.575
Date:                Fri, 22 Sep 2023   Prob (F-statistic):           0.000816
Time:                        15:43:03   Log-Likelihood:                -39.223
No. Observations:                 111   AIC:                             90.45
Df Residuals:                     105   BIC:                             106.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2717      0.076     -3.557      0.0

In [31]:
p_hat = gen_stage1.predict(X)
df['p_hat'] = p_hat

In [32]:
Y = df['q']
X = df[['p_hat','Mon', 'Tue', 'Wed', 'Thu']]
X = sm.add_constant(X)
gen_stage2= sm.OLS(Y,X).fit()
print(gen_stage2.summary())

                            OLS Regression Results                            
Dep. Variable:                      q   R-squared:                       0.193
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     5.034
Date:                Fri, 22 Sep 2023   Prob (F-statistic):           0.000356
Time:                        15:43:04   Log-Likelihood:                -111.90
No. Observations:                 111   AIC:                             235.8
Df Residuals:                     105   BIC:                             252.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.5059      0.161     52.882      0.0

## Wu-Hausman Test

In [33]:
mlr2 = lm.IV2SLS(dependent=df['q'], exog= df[['Mon', 'Tue', 'Wed', 'Thu']], endog=df['p'], instruments=df['Stormy']).fit(cov_type="homoskedastic", debiased=True)
mlr2.wu_hausman()

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 12.5483
P-value: 0.0006
Distributed: F(1,105)
WaldTestStatistic, id: 0x132c999d0

# Trial Models not used in the Final Research Paper

## Stormy as IV + Tue, Wed as controls
- F-stat in stage 1 is lower than our baseline 2SlS model

In [34]:
Z = df['p']
X = df[['Stormy','Tue','Wed']]
X = sm.add_constant(X)
model = sm.OLS(Z,X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     6.805
Date:                Fri, 22 Sep 2023   Prob (F-statistic):           0.000305
Time:                        15:43:08   Log-Likelihood:                -40.470
No. Observations:                 111   AIC:                             88.94
Df Residuals:                     107   BIC:                             99.78
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2869      0.049     -5.889      0.0

In [35]:
p_hat = model.predict(X)
df['p_hat'] = p_hat

In [36]:
Y = df['q']
X = df[['p_hat', 'Tue','Wed']]
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      q   R-squared:                       0.192
Model:                            OLS   Adj. R-squared:                  0.169
Method:                 Least Squares   F-statistic:                     8.449
Date:                Fri, 22 Sep 2023   Prob (F-statistic):           4.33e-05
Time:                        15:43:10   Log-Likelihood:                -112.03
No. Observations:                 111   AIC:                             232.1
Df Residuals:                     107   BIC:                             242.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.5321      0.114     74.716      0.0

## Stormy as IV + Tue as control
- F-stat in stage 1 is lower than our baseline 2SlS model

In [37]:
Z = df['p']
X = df[['Stormy', 'Tue']]
X = sm.add_constant(X)
model = sm.OLS(Z,X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     10.30
Date:                Fri, 22 Sep 2023   Prob (F-statistic):           8.05e-05
Time:                        15:43:12   Log-Likelihood:                -40.472
No. Observations:                 111   AIC:                             86.94
Df Residuals:                     108   BIC:                             95.07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2854      0.043     -6.618      0.0

In [38]:
p_hat = model.predict(X)
df['p_hat'] = p_hat

In [39]:
Y = df['q']
X = df[['p_hat','Tue']]
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      q   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     5.717
Date:                Fri, 22 Sep 2023   Prob (F-statistic):            0.00436
Time:                        15:43:14   Log-Likelihood:                -118.24
No. Observations:                 111   AIC:                             242.5
Df Residuals:                     108   BIC:                             250.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.4041      0.114     74.012      0.0

## Stormy as IV + Wed as Control
- F-stat in stage 1 is lower than our baseline 2SlS model

In [40]:
Z = df['p']
X = df[['Stormy','Wed']]
X = sm.add_constant(X)
model = sm.OLS(Z,X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.144
Method:                 Least Squares   F-statistic:                     10.26
Date:                Fri, 22 Sep 2023   Prob (F-statistic):           8.32e-05
Time:                        15:43:16   Log-Likelihood:                -40.506
No. Observations:                 111   AIC:                             87.01
Df Residuals:                     108   BIC:                             95.14
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2927      0.043     -6.747      0.0

In [41]:
p_hat = model.predict(X)
df['p_hat'] = p_hat

In [42]:
Y = df['q']
X = df[['p_hat','Wed']]
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      q   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.094
Method:                 Least Squares   F-statistic:                     6.714
Date:                Fri, 22 Sep 2023   Prob (F-statistic):            0.00178
Time:                        15:43:17   Log-Likelihood:                -117.32
No. Observations:                 111   AIC:                             240.6
Df Residuals:                     108   BIC:                             248.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.3874      0.113     74.405      0.0

## Model including both Weekdays and Weather as controls for verification
- To verify including Weekdays only is the best model that can strike a balance between generalabiltiy and statistical significance
- With both weekdays and weather, F-stat in stage 1 drops to 3.414 and the p-value of `p_hat` in stage 2 increases

In [43]:
Z = df['p']
X = df[['Stormy', 'Mon', 'Tue', 'Wed', 'Thu', 'Cold', 'Rainy']]
X = sm.add_constant(X)
model = sm.OLS(Z,X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.188
Model:                            OLS   Adj. R-squared:                  0.133
Method:                 Least Squares   F-statistic:                     3.414
Date:                Fri, 22 Sep 2023   Prob (F-statistic):            0.00254
Time:                        15:43:17   Log-Likelihood:                -38.582
No. Observations:                 111   AIC:                             93.16
Df Residuals:                     103   BIC:                             114.8
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2902      0.082     -3.536      0.0

In [44]:
p_hat = model.predict(X)
df['p_hat'] = p_hat

In [45]:
Y = df['q']
X =  df[['p_hat', 'Mon', 'Tue', 'Wed', 'Thu', 'Cold', 'Rainy']]
X = sm.add_constant(X)
model= sm.OLS(Y,X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      q   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     3.579
Date:                Fri, 22 Sep 2023   Prob (F-statistic):            0.00173
Time:                        15:43:19   Log-Likelihood:                -111.74
No. Observations:                 111   AIC:                             239.5
Df Residuals:                     103   BIC:                             261.2
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.4417      0.205     41.211      0.0

## Sargan's test of overidentification

In [46]:
#Overidentification
mlr2 = lm.IV2SLS(dependent=df['q'], exog= None , endog=df['p'], instruments=df[['Stormy', 'Wind']]).fit(cov_type="homoskedastic", debiased=True)
mlr2.sargan

Sargan's test of overidentification
H0: The model is not overidentified.
Statistic: 24.6054
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x133949490

- We reject H0 as p-value <0.05, so having both `Stormy` and `Wind` results in overidentification